<a href="https://colab.research.google.com/github/Mo-null/Data-mining-I/blob/main/Copy_of_Untitled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
importimportimportimport pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Binarizer

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
df = pd.read_csv(url, delimiter=';')
print("Original dataset shape:", df.shape)
df.head()

Original dataset shape: (1599, 12)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [ ]:

target = 'quality'
features = [col for col in df.columns if col != target]

scaler_standard = StandardScaler()
df_standardized = df.copy()
df_standardized[features] = scaler_standard.fit_transform(df[features])

scaler_normal = MinMaxScaler()
df_normalized = df.copy()
df_normalized[features] = scaler_normal.fit_transform(df[features])

print("Standardization applied - Alcohol stats:")
print(f"  Original: mean={df['alcohol'].mean():.2f}, std={df['alcohol'].std():.2f}")
print(f"  Standardized: mean={df_standardized['alcohol'].mean():.2f}, std={df_standardized['alcohol'].std():.2f}")

print("Normalization applied - Alcohol range:")
print(f"  Normalized: min={df_normalized['alcohol'].min():.2f}, max={df_normalized['alcohol'].max():.2f}")

Standardization applied - Alcohol stats:
  Original: mean=10.42, std=1.07
  Standardized: mean=0.00, std=1.00
Normalization applied - Alcohol range:
  Normalized: min=0.00, max=1.00


In [ ]:

df_log = df.copy()
df_log['residual sugar_log'] = np.log1p(df_log['residual sugar'])
df_log['chlorides_log'] = np.log1p(df_log['chlorides'])

df_log['alcohol_sqrt'] = np.sqrt(df_log['alcohol'])

print("Transformations applied:")
print(f"Original residual sugar range: {df['residual sugar'].min():.2f} to {df['residual sugar'].max():.2f}")
print(f"Log transformed range: {df_log['residual sugar_log'].min():.2f} to {df_log['residual sugar_log'].max():.2f}")

Transformations applied:
Original residual sugar range: 0.90 to 15.50
Log transformed range: 0.64 to 2.80


In [ ]:
df_agg = df.copy()
df_agg['quality_category'] = pd.cut(df_agg['quality'], bins=[0, 5, 7, 10], labels=['Low', 'Medium', 'High'])

aggregated = df_agg.groupby('quality_category').agg({
    'alcohol': ['mean', 'std'],
    'pH': ['mean', 'std']
}).round(2)

print("Aggregation by quality category:")
print(aggregated)

Aggregation by quality category:
                 alcohol          pH      
                    mean   std  mean   std
quality_category                          
Low                 9.93  0.76  3.31  0.15
Medium             10.83  1.09  3.31  0.15
High               12.09  1.22  3.27  0.20


/tmp/ipython-input-3667337198.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  aggregated = df_agg.groupby('quality_category').agg({


In [ ]:
df_disc = df.copy()
df_disc['alcohol_level'] = pd.cut(df_disc['alcohol'], bins=3, labels=['Low', 'Medium', 'High'])

binarizer = Binarizer(threshold=df['alcohol'].median())
df_disc['high_alcohol'] = binarizer.fit_transform(df[['alcohol']]).astype(int)

print("Discretization - Alcohol levels:")
print(df_disc['alcohol_level'].value_counts())
print("Binarization - High alcohol:")
print(df_disc['high_alcohol'].value_counts())

Discretization - Alcohol levels:
alcohol_level
Low       985
Medium    559
High       55
Name: count, dtype: int64
Binarization - High alcohol:
high_alcohol
0    842
1    757
Name: count, dtype: int64


In [ ]:
random_sample = df.sample(n=200, random_state=42)

stratified_sample = df.groupby('quality', group_keys=False).apply(lambda x: x.sample(frac=0.2, random_state=42))

systematic_sample = df.iloc[::5]

print("Sampling results:")
print(f"Random sample: {len(random_sample)} rows")
print(f"Stratified sample: {len(stratified_sample)} rows")
print(f"Systematic sample: {len(systematic_sample)} rows")

Sampling results:
Random sample: 200 rows
Stratified sample: 321 rows
Systematic sample: 320 rows


/tmp/ipython-input-2831451803.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stratified_sample = df.groupby('quality', group_keys=False).apply(lambda x: x.sample(frac=0.2, random_state=42))


In [ ]:

df_standardized.to_csv('wine_standardized.csv', index=False)
df_normalized.to_csv('wine_normalized.csv', index=False)
df_log.to_csv('wine_transformed.csv', index=False)

print("All pre-processed datasets saved successfully!")

All pre-processed datasets saved successfully!
